# Advanced Machine Learning Models in PySpark

This notebook implements multiple classification models using PySpark with random search hyperparameter optimization.

## Table of Contents
1. [Importing Libraries](#1.-Importing-Libraries)
2. [Loading and Preprocessing Data](#2.-Loading-and-Preprocessing-Data)
3. [Helper Functions for Evaluation](#3.-Helper-Functions-for-Evaluation-Metrics-and-Visualization)
4. [Logistic Regression with Random Search](#4.-Logistic-Regression-Model-with-Random-Search)
5. [Random Forest with Random Search](#5.-Random-Forest-Model-with-Random-Search)
6. [GBDT with Random Search](#6.-Gradient-Boosted-Decision-Trees-(GBDT)-with-Random-Search)
7. [MLP with Random Search](#7.-Multilayer-Perceptron-(MLP)-with-Random-Search)
8. [Evaluation Metrics and Visualizations](#8.-Evaluation-Metrics-and-Visualizations)
   - [Confusion Matrices](#8.1.-Confusion-Matrices)
   - [Classification Reports](#8.2.-Classification-Reports)
   - [ROC Curves and AUC](#8.3.-ROC-Curves-and-AUC)
   - [Precision-Recall Curves](#8.4.-Precision-Recall-Curves)
9. [Model Comparison](#9.-Model-Comparison)
10. [Test Set Evaluation with Best Model](#10.-Test-Set-Evaluation-with-Best-Model)
11. [Conclusion](#11.-Conclusion)

## 1. Importing Libraries

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D
from math import ceil

# PySpark imports
from pyspark.sql import SparkSession  
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, when, lit, count, lag, expr

# ML imports for classification
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier, MultilayerPerceptronClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.evaluation import MulticlassMetrics

# Scikit-learn imports for metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, precision_recall_curve

# Initialize Spark session
spark = SparkSession.builder.appName("ML Models Spark").getOrCreate()

## 2. Loading and Preprocessing Data

In [ ]:
# File location and type
file_location_train = "dbfs:/FileStore/tables/train_df-2.csv"
file_location_val = "dbfs:/FileStore/tables/val_df.csv"
file_location_test = "dbfs:/FileStore/tables/test_df-2.csv"

train_data = spark.read.csv(file_location_train, header=True, inferSchema=True)
val_data = spark.read.csv(file_location_val, header=True, inferSchema=True)
test_data = spark.read.csv(file_location_test, header=True, inferSchema=True)

In [ ]:
# Select feature columns (all except 'label', 'time', and 'file')
feature_cols = [col for col in train_data.columns if col not in ['label', 'time', 'file']]

# Assemble features into a single vector column
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
train_data = assembler.transform(train_data).select("features", "label")
val_data = assembler.transform(val_data).select("features", "label")
test_data = assembler.transform(test_data).select("features", "label")

# Display the transformed train_data to verify
train_data.show(5)
val_data.show(5)

## 3. Helper Functions for Evaluation Metrics and Visualization

In [ ]:
# Helper function to convert PySpark predictions to numpy arrays for plotting
def get_prediction_labels(predictions_df):
    pred_labels = predictions_df.select("prediction", "label").toPandas()
    y_pred = pred_labels["prediction"].values
    y_true = pred_labels["label"].values
    return y_pred, y_true

# Helper function to get prediction probabilities
def get_prediction_probabilities(predictions_df):
    prob_df = predictions_df.select("probability").toPandas()
    return np.array([x.toArray() for x in prob_df["probability"]])

# Helper function to plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, title="Confusion Matrix"):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

# Helper function to print classification report
def print_classification_report(y_true, y_pred):
    report = classification_report(y_true, y_pred)
    print("Classification Report:")
    print(report)

# Helper function to plot ROC curve for multi-class
def plot_roc_curve(y_true, y_pred_proba, n_classes):
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    # Convert to one-hot encoding for ROC calculation
    y_true_onehot = np.zeros((len(y_true), n_classes))
    for i in range(len(y_true)):
        y_true_onehot[i, int(y_true[i])] = 1
    
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true_onehot[:, i], y_pred_proba[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    # Plot all ROC curves
    plt.figure(figsize=(10, 8))
    colors = ['blue', 'red', 'green', 'orange', 'purple']
    
    for i, color in zip(range(n_classes), colors[:n_classes]):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                 label='ROC curve of class {0} (area = {1:0.2f})'.
                 format(i, roc_auc[i]))
    
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Multi-class ROC Curve')
    plt.legend(loc="lower right")
    plt.show()
    
    # Return the average AUC
    return np.mean(list(roc_auc.values()))

# Helper function to plot precision-recall curve
def plot_precision_recall_curve(y_true, y_pred_proba, n_classes):
    # Compute precision-recall pairs for different probability thresholds
    precision = dict()
    recall = dict()
    avg_precision = dict()
    
    # Convert to one-hot encoding
    y_true_onehot = np.zeros((len(y_true), n_classes))
    for i in range(len(y_true)):
        y_true_onehot[i, int(y_true[i])] = 1
    
    for i in range(n_classes):
        precision[i], recall[i], _ = precision_recall_curve(y_true_onehot[:, i], y_pred_proba[:, i])
        avg_precision[i] = np.mean(precision[i])
    
    # Plot precision-recall curve for each class
    plt.figure(figsize=(10, 8))
    colors = ['blue', 'red', 'green', 'orange', 'purple']
    
    for i, color in zip(range(n_classes), colors[:n_classes]):
        plt.plot(recall[i], precision[i], color=color, lw=2,
                 label='Class {0} (avg precision = {1:0.2f})'.
                 format(i, avg_precision[i]))
    
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Multi-class Precision-Recall Curve')
    plt.legend(loc="best")
    plt.show()
    
    return np.mean(list(avg_precision.values()))

## 4. Logistic Regression Model with Random Search

In [ ]:
# Initialize the Logistic Regression model
log_reg = LogisticRegression(labelCol='label', featuresCol='features', predictionCol='prediction')

# Define the parameter grid for logistic regression
lr_param_grid = ParamGridBuilder() \
    .addGrid(log_reg.regParam, [0.01, 0.1, 1.0, 10.0]) \
    .addGrid(log_reg.elasticNetParam, [0.0, 0.3, 0.7, 1.0]) \
    .addGrid(log_reg.maxIter, [5, 10, 20]) \
    .addGrid(log_reg.family, ["multinomial", "auto"]) \
    .build()

# Initialize the evaluator for F1 score
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='f1')

# Initialize CrossValidator for hyperparameter tuning
lr_cv = CrossValidator(
    estimator=log_reg,
    estimatorParamMaps=lr_param_grid,
    evaluator=evaluator,
    numFolds=3,  # Reduced for faster computation
    parallelism=2  # Parallelism to optimize resource usage
)

# Fit the cross-validator to the training data
print("Training Logistic Regression model with Random Search...")
lr_cv_model = lr_cv.fit(train_data)

# Extract the best model
lr_best_model = lr_cv_model.bestModel

# Print tuned parameters
lr_tuned_params = ['regParam', 'elasticNetParam', 'maxIter', 'family']
lr_best_params = {}

for param in lr_tuned_params:
    if lr_best_model.hasParam(param) and lr_best_model.isSet(getattr(log_reg, param)):
        lr_best_params[param] = lr_best_model.getOrDefault(getattr(log_reg, param))

print("\nBest Logistic Regression Parameters:")
for param, value in lr_best_params.items():
    print(f"  {param}: {value}")

# Make predictions with the best model
lr_train_predictions = lr_best_model.transform(train_data)
lr_val_predictions = lr_best_model.transform(val_data)

# Calculate F1 scores
lr_train_f1 = evaluator.evaluate(lr_train_predictions)
lr_val_f1 = evaluator.evaluate(lr_val_predictions)

print(f"\nLogistic Regression - Training F1 Score: {lr_train_f1:.4f}")
print(f"Logistic Regression - Validation F1 Score: {lr_val_f1:.4f}")

# Extract predictions and probabilities for evaluation
lr_y_pred, lr_y_true = get_prediction_labels(lr_val_predictions)
lr_y_pred_proba = get_prediction_probabilities(lr_val_predictions)

## 5. Random Forest Model with Random Search

In [ ]:
# Initialize Random Forest Classifier
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

# Define parameter grid for Random Forest
rf_param_grid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [50, 100, 200]) \
    .addGrid(rf.maxDepth, [5, 10, 15]) \
    .addGrid(rf.impurity, ["gini", "entropy"]) \
    .addGrid(rf.minInstancesPerNode, [1, 2, 4]) \
    .build()

# Initialize CrossValidator for hyperparameter tuning
rf_cv = CrossValidator(
    estimator=rf,
    estimatorParamMaps=rf_param_grid,
    evaluator=evaluator,
    numFolds=3,  # Reduced for faster computation
    parallelism=2
)

# Fit the cross-validator to the training data
print("\nTraining Random Forest model with Random Search...")
rf_cv_model = rf_cv.fit(train_data)

# Extract the best model
rf_best_model = rf_cv_model.bestModel

# Print tuned parameters
rf_tuned_params = ['numTrees', 'maxDepth', 'impurity', 'minInstancesPerNode']
rf_best_params = {}

for param in rf_tuned_params:
    if rf_best_model.hasParam(param) and rf_best_model.isSet(getattr(rf, param)):
        rf_best_params[param] = rf_best_model.getOrDefault(getattr(rf, param))

print("\nBest Random Forest Parameters:")
for param, value in rf_best_params.items():
    print(f"  {param}: {value}")

# Make predictions with the best model
rf_train_predictions = rf_best_model.transform(train_data)
rf_val_predictions = rf_best_model.transform(val_data)

# Calculate F1 scores
rf_train_f1 = evaluator.evaluate(rf_train_predictions)
rf_val_f1 = evaluator.evaluate(rf_val_predictions)

print(f"\nRandom Forest - Training F1 Score: {rf_train_f1:.4f}")
print(f"Random Forest - Validation F1 Score: {rf_val_f1:.4f}")

# Extract predictions and probabilities for evaluation
rf_y_pred, rf_y_true = get_prediction_labels(rf_val_predictions)
rf_y_pred_proba = get_prediction_probabilities(rf_val_predictions)

## 6. Gradient Boosted Decision Trees (GBDT) with Random Search

In [ ]:
# Initialize GBT Classifier
gbt = GBTClassifier(labelCol="label", featuresCol="features")

# Define parameter grid for GBT
gbt_param_grid = ParamGridBuilder() \
    .addGrid(gbt.maxIter, [10, 20, 50]) \
    .addGrid(gbt.maxDepth, [3, 5, 7]) \
    .addGrid(gbt.stepSize, [0.05, 0.1, 0.2]) \
    .addGrid(gbt.minInstancesPerNode, [1, 2, 4]) \
    .build()

# Initialize CrossValidator for hyperparameter tuning
gbt_cv = CrossValidator(
    estimator=gbt,
    estimatorParamMaps=gbt_param_grid,
    evaluator=evaluator,
    numFolds=3,  # Reduced for faster computation
    parallelism=2
)

# Fit the cross-validator to the training data
print("\nTraining Gradient Boosted Trees model with Random Search...")
gbt_cv_model = gbt_cv.fit(train_data)

# Extract the best model
gbt_best_model = gbt_cv_model.bestModel

# Print tuned parameters
gbt_tuned_params = ['maxIter', 'maxDepth', 'stepSize', 'minInstancesPerNode']
gbt_best_params = {}

for param in gbt_tuned_params:
    if gbt_best_model.hasParam(param) and gbt_best_model.isSet(getattr(gbt, param)):
        gbt_best_params[param] = gbt_best_model.getOrDefault(getattr(gbt, param))

print("\nBest Gradient Boosted Trees Parameters:")
for param, value in gbt_best_params.items():
    print(f"  {param}: {value}")

# Make predictions with the best model
gbt_train_predictions = gbt_best_model.transform(train_data)
gbt_val_predictions = gbt_best_model.transform(val_data)

# Calculate F1 scores
gbt_train_f1 = evaluator.evaluate(gbt_train_predictions)
gbt_val_f1 = evaluator.evaluate(gbt_val_predictions)

print(f"\nGBDT - Training F1 Score: {gbt_train_f1:.4f}")
print(f"GBDT - Validation F1 Score: {gbt_val_f1:.4f}")

# Extract predictions for later evaluation
gbt_y_pred, gbt_y_true = get_prediction_labels(gbt_val_predictions)

## 7. Multilayer Perceptron (MLP) with Random Search

In [ ]:
# Get number of features and classes
num_features = len(train_data.select("features").first()[0])
num_classes = train_data.select("label").distinct().count()

# Define different network architectures for random search
# Format: [input_layer, hidden_layer_1, hidden_layer_2, ..., output_layer]
layers_options = [
    [num_features, num_features, num_classes],  # Simple network
    [num_features, num_features * 2, num_features, num_classes],  # Medium network
    [num_features, num_features * 2, num_features * 2, num_features, num_classes]  # Complex network
]

# Initialize MLP Classifier
# Note: We'll manually implement the random search for MLP since the layers parameter
# requires special handling that isn't well-supported by ParamGridBuilder

# Define parameter combinations
block_sizes = [64, 128, 256]
max_iters = [50, 100, 200]
learning_rates = [0.01, 0.03, 0.1]

# Track best model and score
best_mlp_model = None
best_mlp_val_f1 = 0

print("\nTraining MLP models with Random Search...")
total_combinations = len(layers_options) * len(block_sizes) * len(max_iters) * len(learning_rates)
current_combination = 0

# Manually iterate through parameter combinations
for layers in layers_options:
    for block_size in block_sizes:
        for max_iter in max_iters:
            for step_size in learning_rates:
                current_combination += 1
                print(f"\rTrying combination {current_combination}/{total_combinations}", end="")
                
                # Initialize MLP with current parameters
                mlp = MultilayerPerceptronClassifier(
                    labelCol="label",
                    featuresCol="features",
                    layers=layers,
                    blockSize=block_size,
                    maxIter=max_iter,
                    stepSize=step_size,
                    seed=42
                )
                
                # Train and evaluate the model
                mlp_model = mlp.fit(train_data)
                mlp_val_predictions = mlp_model.transform(val_data)
                mlp_val_f1 = evaluator.evaluate(mlp_val_predictions)
                
                # Update best model if this one is better
                if mlp_val_f1 > best_mlp_val_f1:
                    best_mlp_val_f1 = mlp_val_f1
                    best_mlp_model = mlp_model
                    best_mlp_params = {
                        'layers': layers,
                        'blockSize': block_size,
                        'maxIter': max_iter,
                        'stepSize': step_size
                    }

print("\n\nBest MLP Parameters:")
for param, value in best_mlp_params.items():
    print(f"  {param}: {value}")

# Make predictions with the best model
mlp_train_predictions = best_mlp_model.transform(train_data)
mlp_val_predictions = best_mlp_model.transform(val_data)

# Calculate F1 scores
mlp_train_f1 = evaluator.evaluate(mlp_train_predictions)
mlp_val_f1 = evaluator.evaluate(mlp_val_predictions)

print(f"\nMLP - Training F1 Score: {mlp_train_f1:.4f}")
print(f"MLP - Validation F1 Score: {mlp_val_f1:.4f}")

# Extract predictions for later evaluation
mlp_y_pred, mlp_y_true = get_prediction_labels(mlp_val_predictions)

## 8. Evaluation Metrics and Visualizations

### 8.1. Confusion Matrices

In [ ]:
# Plot confusion matrices for each model
print("Confusion Matrices:")
plot_confusion_matrix(lr_y_true, lr_y_pred, "Logistic Regression Confusion Matrix")
plot_confusion_matrix(rf_y_true, rf_y_pred, "Random Forest Confusion Matrix")
plot_confusion_matrix(gbt_y_true, gbt_y_pred, "GBDT Confusion Matrix")
plot_confusion_matrix(mlp_y_true, mlp_y_pred, "MLP Confusion Matrix")

### 8.2. Classification Reports

In [ ]:
# Print classification reports for each model
print("Logistic Regression Classification Report:")
print_classification_report(lr_y_true, lr_y_pred)

print("\nRandom Forest Classification Report:")
print_classification_report(rf_y_true, rf_y_pred)

print("\nGBDT Classification Report:")
print_classification_report(gbt_y_true, gbt_y_pred)

print("\nMLP Classification Report:")
print_classification_report(mlp_y_true, mlp_y_pred)

### 8.3. ROC Curves and AUC

In [ ]:
# Get number of classes for ROC curves
num_classes = int(train_data.select("label").distinct().count())

# Plot ROC curves
print("ROC Curves and AUC:")
lr_auc = plot_roc_curve(lr_y_true, lr_y_pred_proba, num_classes)
print(f"Logistic Regression Average AUC: {lr_auc:.4f}")

rf_auc = plot_roc_curve(rf_y_true, rf_y_pred_proba, num_classes)
print(f"Random Forest Average AUC: {rf_auc:.4f}")

### 8.4. Precision-Recall Curves

In [ ]:
# Plot Precision-Recall curves
print("Precision-Recall Curves:")
lr_avg_prec = plot_precision_recall_curve(lr_y_true, lr_y_pred_proba, num_classes)
print(f"Logistic Regression Average Precision: {lr_avg_prec:.4f}")

rf_avg_prec = plot_precision_recall_curve(rf_y_true, rf_y_pred_proba, num_classes)
print(f"Random Forest Average Precision: {rf_avg_prec:.4f}")

## 9. Model Comparison

In [ ]:
# Compare all models
models = ["Logistic Regression", "Random Forest", "GBDT", "MLP"]
train_scores = [lr_train_f1, rf_train_f1, gbt_train_f1, mlp_train_f1]
val_scores = [lr_val_f1, rf_val_f1, gbt_val_f1, mlp_val_f1]

# Create a comparison DataFrame
model_comparison = pd.DataFrame({
    'Model': models,
    'Training F1': train_scores,
    'Validation F1': val_scores,
    'Difference (Train-Val)': [train - val for train, val in zip(train_scores, val_scores)]
})

print("Model Performance Comparison:")
print(model_comparison)

# Plot model comparison
plt.figure(figsize=(12, 6))
ind = np.arange(len(models))
width = 0.35

plt.bar(ind - width/2, train_scores, width, label='Training F1')
plt.bar(ind + width/2, val_scores, width, label='Validation F1')

plt.ylabel('F1 Score')
plt.title('Model Comparison')
plt.xticks(ind, models, rotation=15)
plt.legend(loc='best')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Add value labels on top of bars
for i, v in enumerate(train_scores):
    plt.text(i - width/2, v + 0.01, f'{v:.4f}', ha='center')
    
for i, v in enumerate(val_scores):
    plt.text(i + width/2, v + 0.01, f'{v:.4f}', ha='center')

plt.tight_layout()
plt.show()

## 10. Test Set Evaluation with Best Model

In [ ]:
# Find the best model based on validation F1 scores
best_model_index = val_scores.index(max(val_scores))
best_model_name = models[best_model_index]
print(f"Best Model: {best_model_name} with Validation F1: {max(val_scores):.4f}")

# Get the corresponding model object
if best_model_name == "Logistic Regression":
    best_model = lr_best_model
elif best_model_name == "Random Forest":
    best_model = rf_best_model
elif best_model_name == "GBDT":
    best_model = gbt_best_model
elif best_model_name == "MLP":
    best_model = best_mlp_model

# Make predictions on the test set
test_predictions = best_model.transform(test_data)

# Evaluate on test set
test_f1 = evaluator.evaluate(test_predictions)
print(f"Test F1 Score with {best_model_name}: {test_f1:.4f}")

# Get predictions and true labels
test_y_pred, test_y_true = get_prediction_labels(test_predictions)

# Plot confusion matrix for test set
plot_confusion_matrix(test_y_true, test_y_pred, f"{best_model_name} Confusion Matrix (Test Set)")

# Print classification report for test set
print(f"\n{best_model_name} Classification Report (Test Set):")
print_classification_report(test_y_true, test_y_pred)

# If model provides probability predictions, plot ROC and PR curves
if best_model_name in ["Logistic Regression", "Random Forest"]:
    test_y_pred_proba = get_prediction_probabilities(test_predictions)
    
    # Plot ROC curve
    test_auc = plot_roc_curve(test_y_true, test_y_pred_proba, num_classes)
    print(f"{best_model_name} Average AUC (Test Set): {test_auc:.4f}")
    
    # Plot Precision-Recall curve
    test_avg_prec = plot_precision_recall_curve(test_y_true, test_y_pred_proba, num_classes)
    print(f"{best_model_name} Average Precision (Test Set): {test_avg_prec:.4f}")

## 11. Conclusion

In this notebook, we've implemented and evaluated multiple classification models using PySpark with random search hyperparameter optimization:

1. **Logistic Regression**: We tuned parameters including regularization strength, elastic net mixing parameter, maximum iterations, and model family.

2. **Random Forest**: We optimized tree count, maximum depth, impurity measure, and minimum instances per node.

3. **Gradient Boosted Decision Trees (GBDT)**: We fine-tuned maximum iterations, maximum depth, step size, and minimum instances per node.

4. **Multilayer Perceptron (MLP)**: We explored different network architectures, block sizes, maximum iterations, and learning rates.

For each model, we've evaluated:
- Training and validation F1 scores
- Confusion matrices
- Detailed classification reports showing precision, recall, and F1 for each class
- ROC curves and AUC values
- Precision-Recall curves

The best performing model was evaluated on the test set to provide a final assessment of its performance.

Key findings:
1. Random search significantly improved model performance by finding optimal hyperparameters
2. We were able to identify which model architecture works best for this specific classification problem
3. The detailed evaluation metrics provide insights into model behavior across different classes

This comprehensive evaluation allows for informed model selection based on the specific requirements of the classification task.